In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np


In [ ]:
import os
import pandas as pd
from transformers import pipeline
import json
import matplotlib.pyplot as plt
import io
import base64


def process_sentiment(text_list, sent_model):
    output = []
    for text in text_list:
        try:
            analysis = sent_model(text, truncation=True, max_length=512)
            output.append(analysis[0]['label'])
        except Exception as e:
            print(f"Грешка при обработка на текст: {text[:50]}... | Грешка: {e}")
            continue
    return output


def analyze_news_data(input_path, nlp_model, output_path, chunk_size=32):
    if os.path.exists(output_path):
        print(f"Датотеката {output_path} веќе постои. Прескокнување на анализата.")
        return pd.read_csv(output_path)

    df = pd.read_csv(input_path)
    df = df.dropna(subset=['Content'], axis=0).copy()
    df['Content'] = df['Content'].astype(str)
    df = df[df['Content'].str.strip() != ""]

    sent_analyzer = pipeline("sentiment-analysis", model=nlp_model, truncation=True, max_length=512)

    all_sentiments = []
    for i in range(0, len(df), chunk_size):
        chunk = df['Content'][i:i + chunk_size].tolist()
        all_sentiments.extend(process_sentiment(chunk, sent_analyzer))

    df['Sentiment'] = all_sentiments
    df.to_csv(output_path, index=False)
    return df


def create_sentiment_chart(pos, neg, neu):
    oznaki = ['Позитивни', 'Негативни', 'Неутрални']
    vrednosti = [pos, neg, neu]
    boi = ['#2ecc71', '#e74c3c', '#95a5a6']

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(oznaki, vrednosti, color=boi)
    ax.set_ylabel('Вкупно', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7, axis='y')

    for i, v in enumerate(vrednosti):
        ax.text(i, v + 0.5, str(v), ha='center', fontsize=12)

    plt.tight_layout()

    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png', bbox_inches='tight')
    plt.close(fig)
    img_buffer.seek(0)

    return base64.b64encode(img_buffer.read()).decode('utf-8')


def analyze_company_sentiment(df_sentiment, kompanija):
    comp_data = df_sentiment[df_sentiment['Company_code'] == kompanija]

    if comp_data.empty:
        return {"error": f"Нема податоци за компанијата: {kompanija}"}

    sentiments = comp_data['Sentiment']
    stats = sentiments.value_counts()

    poz = stats.get("Positive", 0)
    neg = stats.get("Negative", 0)
    neu = stats.get("Neutral", 0)
    vkupno = sentiments.size

    if vkupno == 0:
        return f"Нема податоци за сентимент анализа за компанијата: {kompanija}"

    poz_proc = (poz / vkupno) * 100
    neg_proc = (neg / vkupno) * 100
    neu_proc = (neu / vkupno) * 100

    if poz_proc > 60:
        signal = "КУПИ"
    elif neg_proc > 60:
        signal = "ПРОДАЈ"
    else:
        signal = "ЧЕКАЈ"

    oznaki = ['Позитивни', 'Негативни', 'Неутрални']
    vrednosti = [poz_proc, neg_proc, neu_proc]
    boi = ['#2ecc71', '#e74c3c', '#95a5a6']

    fig, ax = plt.subplots(figsize=(6, 6))
    wedges, texts, autotexts = ax.pie(
        vrednosti,
        labels=oznaki,
        colors=boi,
        autopct='%1.1f%%',
        startangle=140,
        textprops=dict(color="w")
    )

    ax.axis('equal')
    ax.legend(wedges, oznaki, title="Знаци", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

    pie_buffer = io.BytesIO()
    plt.savefig(pie_buffer, format='png', bbox_inches='tight')
    plt.close(fig)
    pie_buffer.seek(0)

    return {
        "kompanija": kompanija,
        "pozitiven_procent": float(round(poz_proc, 2)),
        "negativen_procent": float(round(neg_proc, 2)),
        "neutralen_procent": float(round(neu_proc, 2)),
        "signal": signal,
        "pita_grafik": base64.b64encode(pie_buffer.read()).decode('utf-8'),
        "bar_grafik": create_sentiment_chart(poz, neg, neu)
    }


def analiziraj_kompanija(kompanija_kod):
    vesti = "news_data.csv"
    sentiment_vesti = "news_sentiment_data.csv"
    nlp_model = "yiyanghkust/finbert-tone"

    df_sentiment = analyze_news_data(vesti, nlp_model, sentiment_vesti)
    rezultat = analyze_company_sentiment(df_sentiment, kompanija_kod)
    return rezultat


if __name__ == "__main__":
    print(analiziraj_kompanija("KMB"))

FileNotFoundError: [Errno 2] No such file or directory: 'news_data.csv'